In [1]:
%%writefile example2.txt
First line
second line
third line
fourth line

Overwriting example2.txt


In [2]:
from pyspark import SparkContext

In [3]:
sc=SparkContext()

In [4]:
sc.textFile('example2.txt')

example2.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

In [5]:
text_rdd=sc.textFile('example2.txt')

In [9]:
words=text_rdd.map(lambda line:line.split())

In [11]:
text_rdd.collect()

['First line', 'second line', 'third line', 'fourth line']

In [12]:
words.collect()

[['First', 'line'], ['second', 'line'], ['third', 'line'], ['fourth', 'line']]

In [14]:
text_rdd.flatMap(lambda line: line.split()).collect()

['First', 'line', 'second', 'line', 'third', 'line', 'fourth', 'line']

In [15]:
%%writefile sevices.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00

Writing sevices.txt


In [17]:
services=sc.textFile('sevices.txt')

In [18]:
services.take(2)

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00']

In [19]:
services.map(lambda x:x.split()).take(3)

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00']]

In [22]:
clean=services.map(lambda x:x[1:] if x[0]=='#' else x)

In [24]:
clean = clean.map(lambda line:line.split())

In [25]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [35]:
pairs=clean.map(lambda lst:(lst[3],lst[-1]))

In [36]:
pairs.collect()

[('State', 'Amount'),
 ('NY', '100.00'),
 ('TX', '450.00'),
 ('CA', '200.00'),
 ('CA', '500.00'),
 ('NY', '750.00'),
 ('TX', '200.00')]

In [42]:
redkey=pairs.reduceByKey(lambda amt1,amt2:float(amt1)+float(amt2))

In [43]:
redkey.collect()

[('State', 'Amount'), ('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [44]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [45]:
#grab state,amount
step1=clean.map(lambda lst: (lst[3],lst[-1]))

In [46]:
#reduce by key
step2=step1.reduceByKey(lambda amt1,amt2:float(amt1)+float(amt2))

In [47]:
#remove title
step3=step2.filter(lambda x:not x[0]=='State')

In [48]:
#sort by amount
step4=step3.sortBy(lambda stamount:stamount[1],ascending=False)
step4.collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]